# Initialize

In [1]:
!pip install -U spacy

You should consider upgrading via the 'c:\users\lhustvei\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.



In [2]:
import spacy
print(spacy.__version__)

2.3.5


In [3]:
# Had to run this to make something below work.
# This is a donwgrade of numpy to a version without a bug.
# This is a temp solution until a new version of numpy has solved the bug.
!pip install numpy==1.19.3 --user

You should consider upgrading via the 'c:\users\lhustvei\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


# Code

In [4]:
def is_float(n):
    try:
        support_float_with_norwegian_format = n.replace(',','.')
        float_n = float(support_float_with_norwegian_format)
    except ValueError:
        return False
    else:
        return True
    
# test
print(is_float('test'))
print(is_float('4'))
print(is_float('4.5'))
print(is_float('4,5'))

False
True
True
True


In [13]:
import spacy
#from spacy.lang.en import English
from spacy.lang.nb import Norwegian
from spacy.matcher import Matcher

# Blank model Norwegian().
# Does not have a POS tagger.
nlp = Norwegian()

#text = "(1) Fartøy med største lengde mellom 10,67 og 15 meter skal ha brannslokningsanlegg."
text = "(4) Fartøy med største lengde mindre enn 10,67 meter bygget etter 1. januar 1992 kan alternativt være utstyrt med brannslokningsanlegg i samsvar med retningslinjer fastsatt i kapittel Y14 i Nordisk Båt Standard (1990)."

doc = nlp(text)

matcher = Matcher(nlp.vocab)

#
# START - spaCy patterns
#

# LENGTH

length_pattern = [
    {"TEXT": {"REGEX": "[0-9]+[,]+[0-9]|[0-9]"}},
    {"LOWER": {"IN": ["meter"]}}]

matcher.add("LENGTH", None, length_pattern)

# LENGTH_PREFIX

length_prefix_pattern_1 = [
    {"LOWER": {"IN": ["mindre"]}},
    {"LOWER": {"IN": ["enn"]}}]

length_prefix_pattern_2 = [
    {"LOWER": {"IN": ["under"]}}]

matcher.add("LENGTH_PREFIX", None, length_prefix_pattern_1, length_prefix_pattern_2)

# WATER_VESSEL

water_vessel_pattern = [
    {"LOWER": {"IN": ["fartøy"]}}]

matcher.add("WATER_VESSEL", None, water_vessel_pattern)

#
# END - spaCy patterns
#

result = []

for match_id, token_start, token_end in matcher(doc):

    match_id_as_string = nlp.vocab.strings[match_id]

    final_token_start = token_start
    final_token_end = token_end

    #
    # Expand result if extra length detected in the words before in the line
    #
    if match_id_as_string == "LENGTH" and token_start > 0:
        prev_token_1 = doc[token_start-1].text
        if prev_token_1 in ("og", "til"):
            prev_token_2 = doc[token_start-2].text
            if is_float(prev_token_2):
                final_token_start = token_start-2

    #
    # convert token_span to char_span.
    # char_span is needed to display correctly withdisplacy.render().
    #
    span = doc[final_token_start:final_token_end]
    span_char_start = span[0].idx
    span_char_end = span[-1].idx + len(span[-1].text)

    # return result
    identified_entity = {'start': span_char_start, 'end': span_char_end, 'label': match_id_as_string}
    result.append(identified_entity)

# display result with spacy styling
display_this_result = {'text': text, 'ents': sorted(result, key=lambda x: x['start']), 'title': 0}
options = {"colors": {"LENGTH": "yellow", "LENGTH_PREFIX": "orange", "WATER_VESSEL": "aquamarine"}}
from spacy import displacy
displacy.render(display_this_result, style='ent', manual=True, options=options)

# Functions run from .py file

In [6]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_vessel_length_overall_no

from spacy_matching_rule_identify_vessel_length_overall_no import identify_length_overall_in_text

importlib.reload(spacy_matching_rule_identify_vessel_length_overall_no)

################
################

identify_length_overall_in_text("(1) Fartøy med største lengde mellom 10,67 og 15 meter skal ha brannslokningsanlegg.")

[{'start': 4, 'end': 10, 'label': 'WATER_VESSEL'},
 {'start': 37, 'end': 54, 'label': 'LENGTH'}]

In [7]:
forward_lines_as_spacy_ner_format = [
  {'text': "(1) Fartøy med største lengde mellom 10,67 og 15 meter skal ha brannslokningsanlegg.", 'ents': [], 'title': 0},
  {'text': '\n', 'ents': [], 'title': 1},
  {'text': 'På fartøy med største lengde 8 til 15 meter skal ovnen kontrolleres hos godkjent foretak innen seks måneder etter at den er installert.', 'ents': [], 'title': 2}]

In [8]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_vessel_length_overall_no

from spacy_matching_rule_identify_vessel_length_overall_no import identify_length_overall_in_text

importlib.reload(spacy_matching_rule_identify_vessel_length_overall_no)

################
################

for line in forward_lines_as_spacy_ner_format:

    text = line['text']

    if text != "\n":
        result_in_a_list = identify_length_overall_in_text(text)

        if result_in_a_list:
            print(result_in_a_list)

[{'start': 4, 'end': 10, 'label': 'WATER_VESSEL'}, {'start': 37, 'end': 54, 'label': 'LENGTH'}]
[{'start': 3, 'end': 9, 'label': 'WATER_VESSEL'}, {'start': 29, 'end': 43, 'label': 'LENGTH'}]


# Merge spacy result with spacy ner formated list of text

In [9]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_vessel_length_overall_no

from spacy_matching_rule_identify_vessel_length_overall_no import merge_spacy_entity_results_to_spacy_ner_format

importlib.reload(spacy_matching_rule_identify_vessel_length_overall_no)

################
################

spacy_ner_formated_text_line = {'text': '(2) Saucer formed rockwool sheets of at least 6 cm thickness shall be provided between smoke uptake and combustible material. Insulation in the form of non-combustible material shall be provided between metal cover plates and combustible material. Alternatively, a pipe with a diameter at least 6 cm larger than the smoke uptake pipe may be fitted outside this pipe, in order to provide vertical air circulation between the pipes. Insulation by saucer formed rockwool or mineral wool twine of at least 3 cm thickness shall be provided between the outer pipe and combustible material. The pipe shall be carried to at least 1 m above the deck, and the outlet shall be protected with a collar.\n',
  'ents': [],
  'title': 53}

spacy_ner_entities_to_be_merged_in_as_a_list = [{'start': 16, 'end': 19, 'label': 'DATE'}, {'start': 6, 'end': 9, 'label': 'DATE'}]

print(merge_spacy_entity_results_to_spacy_ner_format(spacy_ner_formated_text_line,spacy_ner_entities_to_be_merged_in_as_a_list))

spacy_ner_formated_text_line = {'text': '(2) Saucer formed rockwool sheets of at least 6 cm thickness shall be provided between smoke uptake and combustible material. Insulation in the form of non-combustible material shall be provided between metal cover plates and combustible material. Alternatively, a pipe with a diameter at least 6 cm larger than the smoke uptake pipe may be fitted outside this pipe, in order to provide vertical air circulation between the pipes. Insulation by saucer formed rockwool or mineral wool twine of at least 3 cm thickness shall be provided between the outer pipe and combustible material. The pipe shall be carried to at least 1 m above the deck, and the outlet shall be protected with a collar.\n',
  'ents': [{'start': 16, 'end': 19, 'label': 'DATE'}, {'start': 6, 'end': 9, 'label': 'DATE'}],
  'title': 53}

print("")

print(merge_spacy_entity_results_to_spacy_ner_format(spacy_ner_formated_text_line,spacy_ner_entities_to_be_merged_in_as_a_list))

{'text': '(2) Saucer formed rockwool sheets of at least 6 cm thickness shall be provided between smoke uptake and combustible material. Insulation in the form of non-combustible material shall be provided between metal cover plates and combustible material. Alternatively, a pipe with a diameter at least 6 cm larger than the smoke uptake pipe may be fitted outside this pipe, in order to provide vertical air circulation between the pipes. Insulation by saucer formed rockwool or mineral wool twine of at least 3 cm thickness shall be provided between the outer pipe and combustible material. The pipe shall be carried to at least 1 m above the deck, and the outlet shall be protected with a collar.\n', 'ents': [{'start': 6, 'end': 9, 'label': 'DATE'}, {'start': 16, 'end': 19, 'label': 'DATE'}], 'title': 53}

{'text': '(2) Saucer formed rockwool sheets of at least 6 cm thickness shall be provided between smoke uptake and combustible material. Insulation in the form of non-combustible material 

# identify_length_overall_in_text

In [12]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_vessel_length_overall_no

from spacy_matching_rule_identify_vessel_length_overall_no import identify_length_overall_in_spacy_lines

importlib.reload(spacy_matching_rule_identify_vessel_length_overall_no)

################
################

result = identify_length_overall_in_spacy_lines(forward_lines_as_spacy_ner_format)
#print(result)

options = {"colors": {"LENGTH": "yellow", "LENGTH_PREFIX": "orange", "WATER_VESSEL": "aquamarine"}}

# display result with spacy styling
from spacy import displacy
displacy.render(result, style='ent', manual=True, options=options)